In [ ]:
import torch
import torch.nn as nn
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, IterableDataset
from torch.utils.tensorboard import SummaryWriter

In [ ]:
try: import transformers; 
except: 
  !pip install transformers
  import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForTokenClassification, AutoConfig

In [ ]:
class BERT_MLP(nn.Module):
    def __init__(self,num_labels, MODEL_NAME='bert-base-uncased'):
        super().__init__()
        
        self.pretrained_model_config = AutoConfig.from_pretrained(MODEL_NAME)
        self.hidden_dropout_prob = self.pretrained_model_config.hidden_dropout_prob
        self.hidden_size = self.pretrained_model_config.hidden_size 
        
        self.model = AutoModel.from_pretrained(MODEL_NAME, self.pretrained_model_config)
        # self.bert = BertModel.from_pretrained(pre_trained)
        # self.hidden_size = self.bert.config.hidden_size
        
        # define new parameters for classifier
        self.final_hidden = nn.Linear(self.hidden_size,self.hidden_size)
        self.clf = nn.Linear(self.hidden_size, num_labels)
        
        # define variables for loss functions
        self.loss = nn.CrossEntropyLoss()
        
    def cross_entropy(self, logits, labels):
        cl_loss = self.loss(logits, labels)
        
        return cl_loss
        
    def forward(self,input_ids, attention_mask, labels, token_type_ids):
        encoded_layers, pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        encoded_layers = encoded_layers.permute(1, 0, 2)
        
        # enc_hiddens, (last_hidden, last_cell) = self.LSTM(pack_padded_sequence(encoded_layers, inputs[2])) 
        # output_hidden = torch.cat((last_hidden[0], last_hidden[1]), dim=1)
        output_hidden = self.final_hidden(output_hidden)
        output_hidden = torch.tanh(output_hidden)
        output_hidden = F.dropout(output_hidden,self.hidden_dropout_prob)
        
        output = self.clf(output_hidden)
        
        loss = self.cross_entropy(output, labels)
        
        return (output, loss)

In [ ]:
bert_mlp = BERT_MLP(MODEL_NAME='bert-base-uncased', num_labels = 5)

In [2]:
from transformers import BertPreTrainedModel